In [1]:
from dotenv import load_dotenv
import jikanpy
import os
import requests
import pandas as pd
load_dotenv()

secret = os.getenv("secret")
id = os.getenv("id")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_KEY")
jikan = jikanpy.Jikan()

In [2]:
username = 'jackljk'
req = requests.get(f"https://api.myanimelist.net/v2/users/{username}/animelist?limit=1000", headers={"X-MAL-CLIENT-ID": id, "client_secret": secret})

In [3]:
def get_anime_info(anime_id):
    jikan = jikanpy.Jikan()
    anime = jikan.anime(anime_id)['data']
    return anime

In [4]:
df_test = pd.DataFrame()

def test(anime_id):
    test = f"https://api.myanimelist.net/v2/anime/{anime_id}?fields=id,title,main_picture,alternative_titles,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,my_list_status,num_episodes,start_season,broadcast,source,average_episode_duration,rating,pictures,background,related_anime,related_manga,recommendations,studios,statistics'"
    reqtest = requests.get(test, headers={"X-MAL-CLIENT-ID": id, "client_secret": secret})
    
    return reqtest.json()

In [15]:
req = requests.get(f"https://api.myanimelist.net/v2/anime/ranking?ranking_type=airing&limit=10", headers={"X-MAL-CLIENT-ID": id, "client_secret": secret})

In [12]:
x = get_anime_info(58567)

In [61]:
import boto3
import streamlit as st
import pandas as pd
from boto3.dynamodb.types import TypeDeserializer

def get_analytics_data():
    # Connect to dynamodb
    dynamodb = boto3.client(
        "dynamodb",
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name="us-west-2"
    )
    df_dicts = {}
    
    # get top 25 data
    response_top_25 = dynamodb.scan(TableName="anime-dashboard-top-25")
    
    return response_top_25

In [82]:
res = get_analytics_data()
res = res['Items']

In [83]:
def deserialize(data):
    deserializer = TypeDeserializer()
    return {k: deserializer.deserialize(v) for k, v in data.items()}

def parse_analytics_data(data):
    deserialized_data = [deserialize(x) for x in data]
    data = [pd.json_normalize(x, 'data', ['timestamp']) for x in deserialized_data]
    return pd.concat(data, ignore_index=True)


In [86]:
df = parse_analytics_data(res)

In [93]:
df_test = df[['timestamp', 'title_english', 'statistics.members']]
# Convert timestamp to datetime
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

# Pivot the data to have 'title_english' as columns
df_test_pivot = df_test.pivot(index='timestamp', columns='title_english', values='statistics.members')

C:\Users\limja\AppData\Local\Temp\ipykernel_11256\1655815275.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])


In [96]:
!pip install plotly